**1.tmall_order_report.csv**

**1 数据导入与清洗**

In [182]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [183]:
order_path='./tmall_order_report.csv'
data=pd.read_csv(order_path)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28010 entries, 0 to 28009
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   订单编号      28010 non-null  int64  
 1   总金额       28010 non-null  float64
 2   买家实际支付金额  28010 non-null  float64
 3   收货地址      28010 non-null  object 
 4   订单创建时间    28010 non-null  object 
 5   订单付款时间    24087 non-null  object 
 6   退款金额      28010 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.5+ MB


In [184]:
data.head()

,订单编号,总金额,买家实际支付金额,收货地址,订单创建时间,订单付款时间,退款金额
0,1,178.8,0.0,上海,2020-02-21 00:00:00,NaN,0.0
1,2,21.0,21.0,内蒙古自治区,2020-02-20 23:59:54,2020-02-21 00:00:02,0.0
2,3,37.0,0.0,安徽省,2020-02-20 23:59:35,NaN,0.0
3,4,157.0,157.0,湖南省,2020-02-20 23:58:34,2020-02-20 23:58:44,0.0
4,5,64.8,0.0,江苏省,2020-02-20 23:57:04,2020-02-20 23:57:11,64.8


付款时间存在NA值：未付款订单

In [185]:
#列名空格清除
data.columns=data.columns.str.strip()
data.columns

Index(['订单编号', '总金额', '买家实际支付金额', '收货地址', '订单创建时间', '订单付款时间', '退款金额'], dtype='object')

In [186]:
#规范收货地址名称，以便可视化处理
add_city=['北京','上海','天津','重庆']
for a in add_city:
    data['收货地址']=data['收货地址'].str.replace(a,a+'市')
data['收货地址'].unique()

array(['上海市', '内蒙古自治区', '安徽省', '湖南省', '江苏省', '浙江省', '天津市', '北京市', '四川省',
       '贵州省', '辽宁省', '河南省', '广西壮族自治区', '广东省', '福建省', '海南省', '江西省', '甘肃省',
       '河北省', '黑龙江省', '云南省', '重庆市', '山西省', '吉林省', '山东省', '陕西省', '湖北省',
       '青海省', '新疆维吾尔自治区', '宁夏回族自治区', '西藏自治区'], dtype=object)

**2 数据可视化**

In [187]:
result={}
result['总订单数']=data['订单编号'].count()
result['已完成订单数']=data['订单付款时间'][data['订单付款时间'].notnull()].count()
result['未付款订单数']=data['订单编号'][data['订单付款时间'].isnull()].count()
result['退款订单数']=data['退款金额'][data['退款金额']>0].count()
result['总退款金额']=data['退款金额'][data['退款金额']>0].sum()
result['总订单金额']=data['总金额'][data['订单付款时间'].notnull()].sum()
result['总收入金额']=data['买家实际支付金额'].sum()
result

{'总订单数': 28010,
 '已完成订单数': 24087,
 '未付款订单数': 3923,
 '退款订单数': 5646,
 '总退款金额': 572335.92,
 '总订单金额': 2474823.0700000003,
 '总收入金额': 1902487.15}

In [188]:
from pyecharts import options as opts
from pyecharts.charts import Map, Bar, Line
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts,VisualMapOpts
from pyecharts.faker import Faker

**2.1 整体情况总览**

In [189]:
table=Table()
headers=['总订单数', '总订单金额', '已完成订单数', '总实际收入金额', '退款订单数', '总退款金额', '成交率', '退货率']
rows=[
    [
        result['总订单数'],
        f"{result['总订单金额']/10000:.2f}万",
        result['已完成订单数'],
        f"{result['总收入金额']/10000:.2f}万",
        result['退款订单数'],
        f"{result['总退款金额']/10000:.2f}万",
        f"{result['已完成订单数']/result['总订单数']:.2%}",
        f"{result['退款订单数']/result['已完成订单数']:.2%}"
    ]
]
table.add(headers,rows)
table.set_global_opts(
    title_opts=ComponentTitleOpts(title='整体情况总览')
)
table.render_notebook()

总订单数,总订单金额,已完成订单数,总实际收入金额,退款订单数,总退款金额,成交率,退货率
28010,247.48万,24087,190.25万,5646,57.23万,85.99%,23.44%


整体情况的每个单项均可与该商品的月历史数据波动或设定目标进行比较，从而找出可优化点，做出相应调整。
如假设：对比历史数据/目标数据，本月的退货率上升（偏高）/成交率下降（偏低）/总订单数减少等
优化思路：
1.明确对象：对商品分组聚合进行深入分析，确定出现该消极变化的具体类目；
2.分向求因
2.1产品：对对应类目的商品进行价格、质量、流量、舆情等因素的波动监测
2.2顾客：将顾客分类为新客/回头客，分别提取对应数据
2.3店铺：对店铺的推广方式、促销活动、运营热度等因素进行分析
3.筛选真因：多维度分别选取相关性最强的一个或数个因素，提出针对性的改善对策，协调其他部门共同改善
同理，若发生退货率下降/成交率上升/总订单数增加等积极变化，亦可对上述指标进行深入探索，以期将该积极变化映射到其他商品/店铺上。

**2.2 地区分析**

In [190]:
result_area=data[data['订单付款时间'].notnull()].groupby('收货地址').agg({'订单编号':'count'}).to_dict()['订单编号']
area=(
    Map().add('订单量',[*result_area.items()],'china',is_map_symbol_show=False)
    .set_series_opts(label_opts=opts.LabelOpts(is_show=True))
    .set_global_opts(
        title_opts=opts.TitleOpts(title='地区分布'),
        visualmap_opts=opts.VisualMapOpts(max_=2000)
    )
)
area.render('area_analysis.html')

'c:\\Users\\86155\\Desktop\\Tmall\\area_analysis.html'

从地图上可以看出，上海、广东、北京、江苏、浙江、四川省市的订单量位列前茅，而西北地区的订单量则明显处于较低的水平。
其原因可能与发货地、物流便利程度、运费等因素有关，可考虑增加仓库点位、与物流商洽谈合作等。

**2.3 时间分析**

In [191]:
data['订单付款时间']=pd.to_datetime(data['订单付款时间'])
data['订单创建时间']=pd.to_datetime(data['订单创建时间'])

In [192]:
#月订单量走势分析
result_date=data.groupby(data['订单创建时间'].apply(lambda x:x.strftime('%Y-%m-%d'))).agg({'订单编号':'count'}).to_dict()['订单编号']
date=(
    Line()
    .add_xaxis(list(result_date.keys()))
    .add_yaxis('订单量',list(result_date.values()))
    .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(
                    type_='max',name='最大值'
                )
            ]
        )
    )
    .set_global_opts(title_opts=opts.TitleOpts(title='订单量日走势'))
)
date.render('date_analysis.html')

'c:\\Users\\86155\\Desktop\\Tmall\\date_analysis.html'

在2月每日的订单量数据波动中，可以看到17号之前都处于较低的水平，从17号始有了明显的增长趋势，在25号到达了峰值。
其原因可能与疫情封控、物流受限有关。

In [193]:
#日订单量走势分析
result_time=data.groupby(data['订单创建时间'].apply(lambda x:x.strftime('%H'))).agg({'订单编号':'count'}).to_dict()['订单编号']
x=[*result_time.keys()]
y=[*result_time.values()]
time=(
    Bar()
    .add_xaxis(x)
    .add_yaxis('订单量',y)
    .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(name='峰值',type_='max')
            ]
        )
    )
)
time.render('time_analysis.html')

'c:\\Users\\86155\\Desktop\\Tmall\\time_analysis.html'

从每小时的订单量走势来看，一天有3个下单高峰期，分别是10、15、21点，在凌晨5点左右下单量处于低谷期。
可建议客服部门在下单高峰期提高运力，保证回复顾客的及时性，以提高下单量。

In [194]:
#付款时间分析
d=data['订单付款时间']-data['订单创建时间']
d[d.notnull()].apply(lambda x:x.seconds/60).mean()

7.7399046511949745

顾客的平均付款时间为7.7分钟，其波动可能与商品价格、促销活动、商品描述、商品评价等指标有关。
可通过优化商品详情进行改善。

**2.日化.xlsx**

**1 数据导入与清洗**

In [195]:
rihua_path='./日化.xlsx'
order=pd.read_excel(rihua_path,sheet_name='销售订单表')
info=pd.read_excel(rihua_path,sheet_name='商品信息表')
order.head()

,订单编码,订单日期,客户编码,所在区域,所在省份,所在地市,商品编号,订购数量,订购单价,金额
0,D31313,2019-05-16 00:00:00,S22796,东区,浙江省,台州市,X091,892,214,190888.0
1,D21329,2019-05-14 00:00:00,S11460,东区,安徽省,宿州市,X005,276,185,51060.0
2,D22372,2019-08-26 00:00:00,S11101,北区,山西省,忻州市,X078,1450,116,168200.0
3,D31078,2019-04-08 00:00:00,S10902,北区,吉林省,延边朝鲜族自治州,X025,1834,102,187068.0
4,D32470,2019-04-11 00:00:00,S18696,北区,北京市,北京市,X010,887,58,51446.0


In [196]:
info.head()

,商品编号,商品名称,商品小类,商品大类,销售单价
0,X001,商品1,面膜,护肤品,121
1,X002,商品2,面膜,护肤品,141
2,X003,商品3,面膜,护肤品,168
3,X004,商品4,面膜,护肤品,211
4,X005,商品5,面膜,护肤品,185


In [197]:
order.info()
info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31452 entries, 0 to 31451
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   订单编码    31452 non-null  object 
 1   订单日期    31452 non-null  object 
 2   客户编码    31452 non-null  object 
 3   所在区域    31450 non-null  object 
 4   所在省份    31450 non-null  object 
 5   所在地市    31452 non-null  object 
 6   商品编号    31451 non-null  object 
 7   订购数量    31450 non-null  object 
 8   订购单价    31448 non-null  object 
 9   金额      31448 non-null  float64
dtypes: float64(1), object(9)
memory usage: 2.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   商品编号    122 non-null    object
 1   商品名称    122 non-null    object
 2   商品小类    122 non-null    object
 3   商品大类    122 non-null    object
 4   销售单价    122 non-null    int64 
dtypes: int64(1), object(4)
memory usag

order表中“所在区域”、“所在省份”、“商品编号”、“订购数量”、“订购单价”、“金额”列存在空值；info表中无空值。

In [198]:
#查看缺失值
order[order.isnull().any(axis=1)]

,订单编码,订单日期,客户编码,所在区域,所在省份,所在地市,商品编号,订购数量,订购单价,金额
9718,D25844,2019-02-27 00:00:00,S14821,NaN,江苏省,扬州市,X045,538,205,110290.0
9726,D26806,2019-02-23 00:00:00,S16365,NaN,广东省,湛江市,X022,1542,158,243636.0
9781,D22734,2019-02-23 00:00:00,S12453,南区,NaN,广州市,X103,1067,202,215534.0
9978,D26028,2019-07-13 00:00:00,S15878,北区,NaN,北京市,X022,1317,158,208086.0
25029,D22086,2019-09-11 00:00:00,S22887,南区,江西省,九江市,X011,1698,NaN,NaN
25030,D23111,2019-04-22 00:00:00,S10909,东区,浙江省,温州市,X096,804,NaN,NaN
31444,D38486,2019-09-15 00:00:00,S12592,东区,浙江省,杭州市,NaN,NaN,NaN,NaN
31445,D31856,2019-07-12 00:00:00,S17476,东区,湖北省,武汉市,X019,NaN,NaN,NaN


In [199]:
#处理缺失值
order['所在区域'][9718]='东区'
order['所在区域'][9726]='南区'
order['所在省份'][9781]='广东省'
order['所在省份'][9978]='北京市'
order.dropna(inplace=True)

C:\Users\86155\AppData\Local\Temp\ipykernel_8016\2010923823.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order['所在区域'][9718]='东区'
C:\Users\86155\AppData\Local\Temp\ipykernel_8016\2010923823.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order['所在区域'][9726]='南区'
C:\Users\86155\AppData\Local\Temp\ipykernel_8016\2010923823.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order['所在省份'][9781]='广东省'
C:\Users\86155\AppData\Loca

In [200]:
#查看是否存在重复行
order[order.duplicated()].count()

订单编码    6
订单日期    6
客户编码    6
所在区域    6
所在省份    6
所在地市    6
商品编号    6
订购数量    6
订购单价    6
金额      6
dtype: int64

In [201]:
#删除重复行
order.drop_duplicates(inplace=True)

In [202]:
info[info.duplicated()].count()

商品编号    0
商品名称    0
商品小类    0
商品大类    0
销售单价    0
dtype: int64

In [203]:
#对order表重建索引（info表没有删除行数据，无需重建）
order.reset_index(drop=True,inplace=True)

In [204]:
#日期列转换格式
order['订单日期']=order['订单日期'].astype(str).str.replace('#','-')
order['订单日期']=order['订单日期'].apply(lambda x:pd.to_datetime(x,format='%Y-%m-%d'))

In [205]:
#查看订单日期范围
order['订单日期'].min(),order['订单日期'].max()
#清除脏数据
order=order[order['订单日期']<'2021-01-01']

[OUT]: (Timestamp('2019-01-01 00:00:00'), Timestamp('2019-09-30 00:00:00'))
<br>订单日期在2019-01-01到2019-09-30之间。

In [223]:
#各列数据规范化
order['商品编号']=order['商品编号'].str.replace('商品','')
order['客户编码']=order['客户编码'].str.replace('编号','')
order['订单编码']=order['订单编码'].str.replace('订单号','')
order['所在区域']=order['所在区域'].str.strip().replace('男区','南区')
order['订购单价']=order['订购单价'].astype(str).str.replace('元','').astype('int64')
order['订购数量']=order['订购数量'].astype(str).str.replace('个','').astype('int64')
order['金额']=order['金额'].astype(float)
order.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31441 entries, 0 to 31441
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   订单编码    31441 non-null  object        
 1   订单日期    31441 non-null  datetime64[ns]
 2   客户编码    31441 non-null  object        
 3   所在区域    31441 non-null  object        
 4   所在省份    31441 non-null  object        
 5   所在地市    31441 non-null  object        
 6   商品编号    31441 non-null  object        
 7   订购数量    31441 non-null  int64         
 8   订购单价    31441 non-null  int64         
 9   金额      31441 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 2.6+ MB


In [225]:
#关联两个表
total=pd.merge(order,info,on='商品编号',how='inner')
total.head()

,订单编码,订单日期,客户编码,所在区域,所在省份,所在地市,商品编号,订购数量,订购单价,金额,商品名称,商品小类,商品大类,销售单价
0,D31313,2019-05-16,S22796,东区,浙江省,台州市,X091,892,214,190888.0,商品91,粉底,彩妆,214
1,D26674,2019-05-01,S15128,东区,江苏省,南通市,X091,1133,214,242462.0,商品91,粉底,彩妆,214
2,D23381,2019-09-22,S17133,东区,江苏省,宿迁市,X091,1136,214,243104.0,商品91,粉底,彩妆,214
3,D29060,2019-09-10,S14106,东区,江苏省,常州市,X091,544,214,116416.0,商品91,粉底,彩妆,214
4,D21234,2019-07-03,S17197,东区,湖北省,十堰市,X091,342,214,73188.0,商品91,粉底,彩妆,214


注：经对比，订购单价列与销售单价列信息一致，但出于实际场景中该两列信息可能不一致的考虑，此处不作删除。

**2 数据可视化**

**2.1 订单量走势**

In [208]:
r=total.groupby(total['订单日期']).agg({'订单编码':'count'}).to_dict()['订单编码']
r_line=(
    Line()
    .add_xaxis(list(r.keys()))
    .add_yaxis('订单量',list(r.values()))
    .set_global_opts(title_opts=opts.TitleOpts(title='订单量走势'))
)
r_line.render('日化订单量走势.html')

'c:\\Users\\86155\\Desktop\\Tmall\\日化订单量走势.html'

2019年1月至7月订单量呈增长态势，在6-8月达到最高水平，8月底有较明显的下降。

**2.2 购买力TOP10地区**（可对区域/省份/地市进行聚合分析，此处选用地市）

In [213]:
r_cai=total[total['商品大类']=='彩妆'].groupby(total['所在地市']).agg({'订购数量':'sum'}).sort_values(by='订购数量',ascending=False)[:10].to_dict()['订购数量']
r_c=(
    Bar()
    .add_xaxis([*r_cai.keys()])
    .add_yaxis('购买量',[round(v/10000,2) for v in [*r_cai.values()]],label_opts=opts.LabelOpts(position='top',formatter='{@[1]/} 万'))
    .set_global_opts(title_opts=opts.TitleOpts(title='购买力TOP10地区'))
)
r_c.render('彩妆购买力TOP10地区.html')

'c:\\Users\\86155\\Desktop\\Tmall\\彩妆购买力TOP10地区.html'

In [214]:
r_hu=total[total['商品大类']=='护肤品'].groupby(total['所在地市']).agg({'订购数量':'sum'}).sort_values(by='订购数量',ascending=False)[:10].to_dict()['订购数量']
r_h=(
    Bar()
    .add_xaxis([*r_hu.keys()])
    .add_yaxis('购买量',[round(v/10000,2) for v in [*r_hu.values()]],label_opts=opts.LabelOpts(position='top',formatter='{@[1]/} 万'))
    .set_global_opts(title_opts=opts.TitleOpts(title='购买力TOP10地区'))
)
r_h.render('护肤品购买力TOP10地区.html')

'c:\\Users\\86155\\Desktop\\Tmall\\护肤品购买力TOP10地区.html'

彩妆和护肤品购买力的TOP10中城市组成差异不大，排名位次稍有变化。在两个排行榜中，（超）一线城市分别占比80%和70%。

**2.3 美妆类别需求量**

In [221]:
total.groupby(['商品大类','商品小类']).agg({'订购数量':'sum'}).sort_values(by=['商品大类','订购数量'],ascending=[True,False])

订购数量
商品大类 商品小类         
彩妆   口红    2013024
     粉底    1188621
     睫毛膏    587399
     眼影     295795
     蜜粉      45534
护肤品  面膜    5450216
     面霜    4566905
     爽肤水   3525275
     眼霜    3349413
     隔离霜   2488124
     防晒霜   2388610
     洁面乳   1928020

**3 RFM模型建立**

用于量化客户价值，给客户打标签，用以指导二次营销的策略。
<br>R-Recency（最近一次购买时间）
<br>F-Frequency（消费频率）
<br>M-Money（消费金额）
<br>设定打分权重为R-Recency 20%，F-Frequency 30%，M-Money 50%

In [226]:
rfm=total.groupby('客户编码').agg({'订单日期':'max','订单编码':'count','金额':'sum'})
rfm.columns=['最近一次购买时间','消费频率','消费金额']
#添加百分比排名
rfm['R']=rfm['最近一次购买时间'].rank(pct=True)
rfm['F']=rfm['消费频率'].rank(pct=True)
rfm['M']=rfm['消费金额'].rank(pct=True)
rfm['Score']=round(rfm['R']*20+rfm['F']*30+rfm['M']*50,1)
rfm.sort_values(by='Score',ascending=False)


,最近一次购买时间,消费频率,消费金额,R,F,M,Score
客户编码,,,,,,,
S17476,2019-09-30,68,10258002.0,0.980148,0.985226,0.987073,98.5
S22326,2019-09-30,62,10074609.0,0.980148,0.973223,0.984303,98.0
S11581,2019-09-28,79,10333668.0,0.918283,0.996768,0.987996,97.7
S12848,2019-09-29,66,9673572.0,0.944598,0.980609,0.980609,97.3
S19095,2019-09-26,81,11031632.0,0.864728,0.999077,0.996307,97.1
...,...,...,...,...,...,...,...
S12690,2019-05-07,7,917233.0,0.012927,0.022622,0.024931,2.2
S11176,2019-06-09,7,614134.0,0.036011,0.022622,0.009234,1.9
S18379,2019-07-05,4,400195.0,0.071099,0.003232,0.004617,1.7


可将80分以上的客户标记为优质客户，在资源有限时，可以优先服务好优质客户。